In [3]:
import os
from tqdm import tqdm
import numpy as np
from load_dataset import load_corpus, load_queries, load_retrieval
from crossencoder_bm25 import CustomCrossEncoder
import pickle


On lit les passages retrieve par BM25 (top 1000) :

In [4]:
data_folder = "./data/msmarco-passage/"
corpus = load_corpus(os.path.join(data_folder, "collection.tsv"))
queries = load_queries(os.path.join(data_folder, "queries.dev.small.tsv"))
retrieval_samples = load_retrieval(
        os.path.join(data_folder, "msmarco.bm25.dev.small.tsv.gz"), corpus, queries
    )


8.84Mit [00:14, 593kit/s]
6.98kit [00:00, 1.22Mit/s]
6.97Mit [00:26, 261kit/s] 


Initialisation du Cross-Encoder (CEBM25CAT) :

In [3]:
model_name = "output/training_ms-marco_cross-encoder-microsoft-MiniLM-L12-H384-uncased-2023-05-20_12-13-19-latest"
model = CustomCrossEncoder(model_name)

Re-Ranking :

In [4]:
similarity_scores = {}
for qid in tqdm(queries):
    similarity_scores[qid] = [model.predict(retrieval_samples[qid])]
    

100%|██████████| 6980/6980 [1:49:00<00:00,  1.07it/s]


On save parce que c'est long :

In [5]:
if not os.path.exists("reranking.pkl"):
    with open('reranking.pkl', 'wb') as f:
        pickle.dump(similarity_scores, f)
else:
    with open('reranking.pkl', 'rb') as f:
        similarity_scores = pickle.load(f)

Maintenant on prépare les données pour pouvoir calculer les métriques...

In [8]:
dev_filepath = "runs/run.msmarco-passage.bm25tuned.txt"
dev = {}

with open(dev_filepath, "r") as fIn:
    for line in tqdm(fIn, unit_scale=True):
        qid, corpus_id, rank = line.strip().split("\t")
        
        if qid in dev:
            dev[qid].append([qid, corpus_id])
        else:
            dev[qid] = [[qid, corpus_id]]



6.97Mit [00:14, 487kit/s] 


Il ne faut pas oublier de sort :

In [9]:
results = [np.concatenate((np.array(dev[qid]), similarity_scores[qid][0].reshape(-1, 1)), axis=1) for qid in queries]


In [10]:
results[0]

array([['1048585', '7187158', '0.9928669'],
       ['1048585', '7187157', '0.9925741'],
       ['1048585', '7187163', '0.9899811'],
       ...,
       ['1048585', '5771111', '0.00022678288'],
       ['1048585', '6073381', '0.00023700217'],
       ['1048585', '6339403', '0.0002411898']], dtype='<U32')

In [11]:
results_sorted = [results[i][results[i][:, 2].argsort()[::-1]] for i in range(len(results))]

In [12]:
results_sorted[0]

array([['1048585', '7187158', '0.9928669'],
       ['1048585', '7187157', '0.9925741'],
       ['1048585', '7187160', '0.99153566'],
       ...,
       ['1048585', '4404232', '0.00022478984'],
       ['1048585', '4971866', '0.00022478664'],
       ['1048585', '6091438', '0.00022475321']], dtype='<U32')

Et on peut enfin sauvegarder les résultats pour Pyserini :

In [14]:
import csv

with open("run.msmarco-passage.reranking.txt", "w") as fOut:
    writer = csv.writer(fOut, delimiter="\t", lineterminator="\n")
    for result in results_sorted:
        for rank, line in enumerate(result):
            qid = line[0]
            corpus_id = line[1]
            writer.writerow([qid, corpus_id, rank + 1])